In [13]:
from datetime import datetime
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder

import xgboost as xgb

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler

import skforecast
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from skforecast.utils import save_forecaster
from skforecast.utils import load_forecaster
import shap


In [14]:
df = pd.read_csv('../data/Top100_prediction_con.csv')
print(df.shape)
df.sample(1)

(1739323, 12)


,Unnamed: 0,id,store,date,sales,item,store_code,yearweek,sell_price,weekday,event,income
919594,33166347,SUPERMARKET_3_150_PHI_1,Midtown_Village,2014-01-20,3,SUPERMARKET_3_150,PHI_1,201402,2.7,Monday,0.0,8.1


In [15]:
lista_productos100 = list(df['item'].unique())

In [16]:
resultados = []

# Copia del DataFrame original
df_original = df.copy()

# Seleccionar los primeros 5 productos únicos
productos = list(df_original['item'].unique())

for i in lista_productos100:
        dataset = df_original[['item','date','sales','weekday','event']]
        df_product = dataset[dataset['item']==i]
        df_product_sin_duplicates = df_product[['date','weekday', 'event']].drop_duplicates()
        ts_predict = df_product.groupby(['date'])['sales'].sum().reset_index()
        ts_predict = ts_predict.merge(df_product_sin_duplicates, on=['date'], how='left')
        ts_predict = pd.get_dummies(data=ts_predict, columns=['weekday'], dtype=int)
        ts_predict['date'] = pd.to_datetime(ts_predict['date'])
        ts_predict.sort_values('date', ascending=True, inplace=True)
        ts_predict.set_index('date', inplace=True)
        steps = 30
        ts_predict = ts_predict.asfreq('D')
        y, exog = ts_predict['sales'], ts_predict.drop(columns=['sales'])
        y_train, y_test = y[:-30], y[-30:]
        exog_train, exog_test = exog[:-30], exog[-30:]        
        forecaster = ForecasterAutoreg(
                    regressor = RandomForestRegressor(random_state=123,
                                                        max_depth=None,
                                                        min_samples_leaf=2, 
                                                        min_samples_split=2,
                                                        n_estimators=50),
                    lags      =  14
                )
        
        start_date = '2016-04-25' 
        end_date ='2016-05-30'
        # Crear un rango de fechas
        date_range = pd.date_range(start=start_date, end=end_date)
        # Crear el dataframe
        df1 = pd.DataFrame(date_range, columns=['date'])
        df1['date'] = pd.to_datetime(df1['date'])
        df1['weekday'] = df1['date'].dt.day_name()
        df1['event'] = 0
        df1 = pd.get_dummies(data=df1, columns=['weekday'], dtype=int)
        df1.sort_values('date', inplace=True, ascending=True)
        df1.set_index('date', inplace=True)
        df1 = df1.asfreq('D')
        exog_test = pd.concat([exog_test, df1], ignore_index=False)
        forecaster.fit(y=y_train, exog=exog_train)
        predictions = forecaster.predict(steps=60, exog=exog_test).to_list()
        resultados.append({'i':i, 'prediction':predictions,'test':y_test.values})

/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider 

In [17]:
tiendas_inventario = pd.DataFrame(resultados)

In [20]:
tiendas_inventario[['i','test']]

,i,test
0,ACCESORIES_1_108,"[10, 13, 4, 10, 5, 13, 12, 12, 18, 7, 10, 5, 3..."
1,HOME_&_GARDEN_1_027,"[29, 23, 26, 20, 17, 26, 20, 31, 23, 23, 18, 2..."
2,HOME_&_GARDEN_1_053,"[43, 47, 40, 21, 22, 27, 42, 46, 57, 27, 32, 2..."
3,HOME_&_GARDEN_1_140,"[16, 22, 7, 9, 19, 14, 17, 22, 25, 10, 12, 11,..."
4,HOME_&_GARDEN_1_177,"[38, 41, 25, 30, 21, 28, 37, 59, 68, 52, 43, 1..."
...,...,...
95,SUPERMARKET_3_499,"[58, 38, 59, 71, 92, 97, 116, 147, 122, 160, 1..."
96,ACCESORIES_1_158,"[72, 56, 64, 52, 55, 53, 56, 41, 77, 55, 57, 5..."
97,SUPERMARKET_3_282,"[275, 296, 182, 152, 162, 153, 203, 289, 336, ..."
98,ACCESORIES_1_354,"[70, 45, 44, 45, 47, 32, 50, 61, 69, 49, 49, 5..."


In [ ]:
tiendas_inventario['test_suma'] = tiendas_inventario['test'].apply(sum)
tiendas_inventario

In [22]:
suma = tiendas_inventario['test_suma'].sum()

In [23]:
suma

240920

In [24]:
tiendas_inventario['prediction'] = tiendas_inventario['prediction'].apply(lambda x: np.round(x, decimals=0))

In [27]:
tiendas_inventario['tiendas_inventario_test'] = tiendas_inventario['prediction'].apply(lambda x: sum(x[:30]))
tiendas_inventario['tiendas_inventario_prediction'] = tiendas_inventario['prediction'].apply(lambda x: sum(x[30:]))

In [29]:
tiendas_inventario[['i','test_suma','tiendas_inventario_test','tiendas_inventario_prediction']]

,i,test_suma,tiendas_inventario_test,tiendas_inventario_prediction
0,ACCESORIES_1_108,266,293.0,221.0
1,HOME_&_GARDEN_1_027,624,683.0,703.0
2,HOME_&_GARDEN_1_053,1012,909.0,836.0
3,HOME_&_GARDEN_1_140,485,437.0,397.0
4,HOME_&_GARDEN_1_177,477,892.0,775.0
...,...,...,...,...
95,SUPERMARKET_3_499,3379,2829.0,3190.0
96,ACCESORIES_1_158,1633,1509.0,1381.0
97,SUPERMARKET_3_282,6010,5594.0,5289.0
98,ACCESORIES_1_354,1602,1672.0,1657.0


In [34]:
total_inventario_productos_real = tiendas_inventario['test_suma'].sum()
total_inventario_productos_predicho = tiendas_inventario['tiendas_inventario_test'].sum()
print(total_inventario_productos_real)
print(total_inventario_productos_predicho)
print(total_inventario_productos_real - total_inventario_productos_predicho)

240920
239651.0


: 

In [33]:
total_inventario_productos_real - total_inventario_productos_predicho

1269.0

## PREDICCION

In [1]:
resultados = []

# Copia del DataFrame original
df_original = df.copy()

# Seleccionar los primeros 5 productos únicos
productos = list(df_original['item'].unique())

for i in lista_productos100:
        dataset = df_original[['item','date','sales','weekday','event']]
        df_product = dataset[dataset['item']==i]
        df_product_sin_duplicates = df_product[['date','weekday', 'event']].drop_duplicates()
        ts_predict = df_product.groupby(['date'])['sales'].sum().reset_index()
        ts_predict = ts_predict.merge(df_product_sin_duplicates, on=['date'], how='left')
        ts_predict = pd.get_dummies(data=ts_predict, columns=['weekday'], dtype=int)
        ts_predict['date'] = pd.to_datetime(ts_predict['date'])
        ts_predict.sort_values('date', ascending=True, inplace=True)
        ts_predict.set_index('date', inplace=True)
        ts_predict = ts_predict.asfreq('D')
        y, exog = ts_predict['sales'], ts_predict.drop(columns=['sales'])
        y_train, y_test = y[:-30], y[-30:]
        exog_train, exog_test = exog[:-30], exog[-30:]
        
        forecaster = ForecasterAutoreg(
                    regressor = RandomForestRegressor(random_state=123,
                                                        max_depth=None,
                                                        min_samples_leaf=2, 
                                                        min_samples_split=2,
                                                        n_estimators=50),
                    lags      =  14
                )
        
        start_date = '2016-04-25' 
        end_date ='2016-05-30'
        # Crear un rango de fechas
        date_range = pd.date_range(start=start_date, end=end_date)
        # Crear el dataframe
        df1 = pd.DataFrame(date_range, columns=['date'])
        df1['date'] = pd.to_datetime(df1['date'])
        df1['weekday'] = df1['date'].dt.day_name()
        df1['event'] = 0
        df1 = pd.get_dummies(data=df1, columns=['weekday'], dtype=int)
        df1.sort_values('date', inplace=True, ascending=True)
        df1.set_index('date', inplace=True)
        df1 = df1.asfreq('D')
        exog_test = pd.concat([exog_test, df1], ignore_index=False)

        forecaster.fit(y=y_train, exog=exog_train)
        predictions = forecaster.predict(steps=60, exog=exog_test).to_list()
        resultados.append({'i':i, 'prediction':predictions,'test':y_test.values})

NameError: name 'df' is not defined

In [16]:
pd.DataFrame(resultados)

,i,prediction,test
0,ACCESORIES_1_108,"[10.923238095238096, 10.905142857142856, 11.17...","[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [1...."
1,HOME_&_GARDEN_1_027,"[28.29742857142857, 27.403476190476194, 22.649...","[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [1...."
2,HOME_&_GARDEN_1_053,"[36.19519841269842, 39.81838095238094, 29.6421...","[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [1...."
3,HOME_&_GARDEN_1_140,"[17.884714285714278, 18.459428571428568, 13.71...","[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [1...."
4,HOME_&_GARDEN_1_177,"[34.91716666666666, 42.13176190476191, 29.9955...","[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [1...."
...,...,...,...
95,SUPERMARKET_3_499,"[76.10561904761906, 71.27733333333336, 85.6287...","[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [1...."
96,ACCESORIES_1_158,"[51.30011904761905, 56.08266666666666, 50.5970...","[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [1...."
97,SUPERMARKET_3_282,"[220.25947619047616, 245.74099999999999, 177.0...","[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [1...."
98,ACCESORIES_1_354,"[62.202055555555546, 64.70076190476189, 55.185...","[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [1...."


In [14]:
tiendas = pd.DataFrame(resultados)
tiendas.head(1)

,i,prediction
0,ACCESORIES_1_108,"[10.923238095238096, 10.905142857142856, 11.17..."


In [17]:
file_name = "productos_global_unidades_vendidas.xlsx"
tiendas.to_excel(file_name)